In [ ]:
import requests
import itertools

문자열 = '0123456789'  # 특수 문자는 없다고 가정한다

for 패스워드길이 in range(1, 5):  # 1 - 1~3에서 1~5까지 해볼 것
    for password in itertools.product(문자열, repeat=패스워드길이):
        pw = ''.join(password)
        print(pw)
        로그인패킷 = {'id': 'hojun', 'pw': pw}
        print(로그인패킷)
        
        address = requests.post('http://127.0.0.1:8000/admin/', data=로그인패킷)
        if 'Wellcome' in address.text:
            exit()

In [ ]:
import requests
import itertools

문자열 = '0123456789orem Ipsum is simply dummy text of the printi'

for length in range(1, 5):
    for password in itertools.product(문자열, repeat=length):
        pw = ''.join(password)
        print(pw)

In [ ]:
import itertools


list(itertools.product('ABCD', repeat=2))

In [ ]:
a = list(itertools.combinations('ABCD', 2))
len(a)

In [ ]:
from http.server import HTTPServer, BaseHTTPRequestHandler
import urllib.parse as parser


class ServerHandler(BaseHTTPRequestHandler):

    def do_GET(self):
        self.send_response(200)
        self.send_header('content-type', 'text/html')
        self.end_headers()
        print('get 요청이 들어왔습니다.')

        f = open("index.html", 'r', encoding='UTF8')
        data = f.read()
        # print(data)
        f.close()
        self.wfile.write(data.encode())

    def do_POST(self):
        self.send_response(200)
        self.send_header('content-type', 'text/html')
        self.end_headers()
        print('post 요청이 들어왔습니다.')
        data = self.rfile.read(int(self.headers['Content-Length']))
        if data is not None:
            data_decode = dict(parser.parse_qs(data.decode()))
        if data_decode['id'] == ['hojun'] and data_decode['pw'] == ['1234']:
            f = open("success.html", 'r', encoding='UTF8')
            data = f.read()
            f.close()
            self.wfile.write(data.encode())
        else:
            f = open("index_fail.html", 'r', encoding='UTF8')
            data = f.read()
            f.close()
            self.wfile.write(data.encode())
        print(f'post params = {data_decode}')


PORT = 8080
server = HTTPServer(('', PORT), ServerHandler)
print(f'서버가 {PORT}로 서비스 되고 있습니다.')
server.serve_forever()

mkdir modeltest
cd modeltest
python -m venv venv

# 가상환경속으로 들어가기
.\venv\Scripts\activate # window
.\venv\Script\activate.bat # window
source ./venv/bin/activate # mac, linux

pip install django
django-admin startproject project .
python manage.py migrate

# settings.py에서 접속할 수 있는 사람 설정
ALLOWED_HOSTS = ['*'] # 28번째 줄에 접속할 수 있는 사람을 모든 사람으로 변경


python manage.py startapp main


INSTALLED_APPS = [
    'django.contrib.admin',
    'django.contrib.auth',
    'django.contrib.contenttypes',
    'django.contrib.sessions',
    'django.contrib.messages',
    'django.contrib.staticfiles',
    'main',
]

##############################

from django.db import models

class Post(models.Model):
    title = models.CharField(max_length=100)
    content = models.TextField()
    
class Comment(models.Model):
    post = models.ForeignKey(Post, on_delete=models.CASCADE)
    text = models.TextField()

##############################

python manage.py makemigrations
python manage.py migrate

##############################

python manage.py shell

##############################

from main.models import Post, Comment

Post.objects.create(title='sample1', content='sample11').save()
Post.objects.create(title='sample2', content='sample22').save()

a = Post.objects.get(pk=1)
Comment.objects.create(post=a, text='sample1').save()
Comment.objects.create(post=a, text='sample2').save()

b = Post.objects.get(pk=2)
Comment.objects.create(post=b, text='sample11').save()
Comment.objects.create(post=b, text='sample22').save()
Comment.objects.create(post=b, text='sample33').save()

Comment.objects.all() # 5개가 나옴, 그래서 이것을 모두 순회를 도는 것은 적절치 않음

# posts = Post.objects.all()
# 
# for post in posts:
#       comment = Comment.objects.filter(post=post)
#     # Comment.objects.filter(post=a)와 같이 순회를 도는데 4개를 모두 조회해야 함
#     # 그런데 만약 4개가 아니라 100만개라면?

# 아래와 같이 해결 가능
# posts = Post.objects.prefetch_related('comment_set').all()
# 
# for post in posts:
#     comments = post.comment_set.all()
#     for comment in comments:
#         # 여기서 comment 처리

# 실습
posts = Post.objects.prefetch_related('comment_set').all()

for post in posts:
    print(post.comment_set.all())

# annotate 실습
from django.db.models import Count

# post에 comment_count라는 속성이 없는데 임시로 추가할 수 있습니다.
# DB에는 반영되지 않습니다.
posts_with_comment_count = Post.objects.annotate(comment_count=Count('comment'))

for post in posts_with_comment_count:
    print(f"포스트 제목: {post.title}, 댓글 수: {post.comment_count}")


# aggregate 실습
total_comment_count = Comment.objects.aggregate(total=Count('id'))
print(f"전체 댓글 수: {total_comment_count['total']}")
# 전체 댓글 수: 4


from django.db.models import Avg
average_comment_per_post = Post.objects.annotate(comment_count=Count('comment')).aggregate(average=Avg('comment_count'))

print(f"포스트 당 평균 댓글 수: {average_comment_per_post['average']}")
# 포스트 당 평균 댓글 수: 2.0